In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
from operator import itemgetter
import seaborn as sns



In [14]:
def Dice_Calculator(file1, file2):
    img1 = nib.load(file1)
    img1_data = img1.get_fdata()
    hdr1 = img1.header
    img1_flat = img1_data.flatten()
    img1_flat = [int(i) for i in img1_flat]

    img2 = nib.load(file2)
    img2_data = img2.get_fdata()
    hdr2 = img2.header
    img2_flat = img2_data.flatten()
    img2_flat = [int(i) for i in img2_flat]
    
    values1 = np.unique(img1_flat)
    values2 = np.unique(img2_flat)
    counts1 = np.array(range(1, len(img1_flat)+1))
    counts2 = np.array(range(1, len(img2_flat)+1))
    counts1 = [int(i) for i in counts1]
    counts2 = [int(i) for i in counts2]

    img1_flatten = list(zip(img1_flat, counts1))
    img2_flatten = list(zip(img2_flat, counts2))

    img1_flatten = sorted(img1_flatten, key = lambda l:l[0])
    img2_flatten = sorted(img2_flatten, key = lambda l:l[0])
    
    img1_cell = [[] for i in range(len(values1))]
    img2_cell = [[] for i in range(len(values2))]
    start = 0
    for i in range(0,len(values1)-1):
        img1_list = [x[0] for x in img1_flatten]
        end = max(loc for loc, val in enumerate(img1_list) if val == values1[i])
        for j in range(start,end):
            img1_cell[i].append(img1_flatten[j][1])  
        start = end+1

    start = 0
    for i in range(0,len(values2)-1):
        img2_list = [x[0] for x in img2_flatten]
        end = max(loc for loc, val in enumerate(img2_list) if val == values2[i])
        for j in range(start,end):
            img2_cell[i].append(img2_flatten[j][1])  
        start = end+1
        
    Dice_scores = np.zeros((len(values1),len(values2)),dtype=float)
    for i in range(0,len(values1)-1):
        for j in range(0,len(values2)-1):
            sims = len(set(img1_cell[i]) & set(img2_cell[j]))
            Dice_scores[i][j] = 2*sims/(len(img1_cell[i])+len(img2_cell[j]))
    
    mean_Dice = Dice_scores.mean()
    return mean_Dice

In [ ]:
mypath = '../Nifti_images'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
Dice_scores = np.zeros((len(files)-1,len(files)-1),dtype=float)

for i in range(0,len(files)-2):
    for j in range(i,len(files)-1):
        image1_name = files[i]
        filename1 = join(mypath, image1_name)
        image2_name = files[j]
        filename2 = join(mypath, image2_name)

        Dice_scores[i][j] = Dice_Calculator(filename1, filename2)

sns.heatmap(Dice_scores)

0.008982353023138632